### Grouping, Joining and Sorting 

A company might want to find out its total sales in the last quarter. But it might want to find out which countries have done particularly well (or poorly). Or perhaps the head of sales would like to see how much each individual salesperson has brought in, or how much each product has contributed to the company’s income.

These types of questions can be answered using a technique known as "grouping." Much like the GROUP BY clause in an SQL query, we can use grouping in pandas to ask the same question for various subsets of our data.

Another common technique, also encountered when working with SQL databases, is that of "joining." What happens if the data you want to analyze is split across two separate data frames? For example, one data frame lists each sales region and that region’s manager, while a second data frame contains this quarter’s regional sales results. If you want to show the manager’s name alongside the sales results, you’ll want to join the data frames together, then



TODO: 
- Load the CSV file into a data frame, using only the columns passenger_count, trip_distance, and total_amount.
- Using a descending sort, find the average cost of the 20 longest (in distance) taxi rides in January 2019.
- Now using an ascending sort, find the average cost of the 20 longest (in distance) taxi rides in January 2019. Specify "mergesort" as the sorting algorithm. Are the results any different?
- Sort by ascending passenger count and descending trip distance. (So we’ll start with the longest trip with 0 passengers and end with the shortest trip with 6 passengers.) What is the average price paid for the top 50 rides?

In [76]:
import pandas as pd

taxi_df = pd.read_csv('data/nyc_taxi_2020-01.csv', usecols = ['passenger_count', 'total_amount', 'trip_distance'])

taxi_df.head()

,passenger_count,trip_distance,total_amount
0,1.0,1.2,11.27
1,1.0,1.2,12.30
2,1.0,0.6,10.80
3,1.0,0.8,8.16
4,1.0,0.0,4.80


In this exercise, we again want to sort the rows of our data frame—but we want to do it based on the values in a particular column, rather than the index. You could argue that there isn’t really much difference between the two; we could take a column, temporarily make it the index, sort by the index, and then return the column back to the data frame. But the difference between sort_index and sort_values isn’t just technical. We’re thinking about our data, and how we want to access it, in different ways.

sort_values is also different from sort_index in another way, namely that we can sort by any number of columns. Imagine, once again, that your data frame contains sales data. You might want to sort it by price, by region, or by salesperson—or even by a combination of these. When we sort by the index, by contrast, we’re effectively sorting by a single column.

In [17]:
# Sort and find the mean of the top 20 rides in January

taxi_df.sort_values(by = 'trip_distance', ascending = False)["trip_distance"].iloc[:20].mean()

10650.013000000003

In [16]:
# Using mergesort to sort data
taxi_df.sort_values(by = 'trip_distance', kind = 'mergesort')["trip_distance"].iloc[ -20: ].mean() 

10650.013

In [18]:
# Sort by passenger_count in ascending and trip_distance in descending order

taxi_df.sort_values(by = ['passenger_count', 'trip_distance'], ascending = [True, False])["total_amount"].iloc[:50].mean()

147.06880000000004

TODO
- In which five rides did people pay the most per mile? How far did people go on those trips?

In [73]:
import numpy as np

# Drop NAs 
cost_df = taxi_df[['trip_distance', "total_amount", "passenger_count"]].dropna()

# Remove 0's from Trip Distance and Total Amount 
cost_df = cost_df.loc[((cost_df['trip_distance'] > 0) & (cost_df['total_amount'] > 0) & (cost_df['passenger_count'] > 1))]

# Create the cost_per_mile column
cost_df["cost_per_mile"] = (cost_df["total_amount"] / cost_df["trip_distance"])

# Sort Values by cost_per_mile and 
cost_df.sort_values(by = 'cost_per_mile', ascending = False)[["cost_per_mile", "total_amount", "trip_distance"]].iloc[:5]

,cost_per_mile,total_amount,trip_distance
3802387,14496.0,144.96,0.01
3837826,13296.0,132.96,0.01
6046285,12096.0,120.96,0.01
5176322,11556.0,115.56,0.01
6008074,11436.0,114.36,0.01


TODO 
- Let’s assume that multi-passenger rides are split evenly among the passengers. Given that assumption, in which 10 multi-passenger rides did each individual pay the greatest amount?

In [74]:
# Calculate the cost_per_mile_per_passenger
cost_df["cost_per_mile_per_passenger"] = (cost_df["cost_per_mile"] / cost_df["passenger_count"])

# Sort in descending order of multi-passenger rides
cost_df.sort_values(by = 'cost_per_mile_per_passenger', ascending = False)[["cost_per_mile_per_passenger", "cost_per_mile", "passenger_count","total_amount", "trip_distance"]].iloc[:10]

,cost_per_mile_per_passenger,cost_per_mile,passenger_count,total_amount,trip_distance
3802387,7248.0,14496.0,2.0,144.96,0.01
3837826,6648.0,13296.0,2.0,132.96,0.01
1141997,5418.0,10836.0,2.0,108.36,0.01
5202479,5115.0,10230.0,2.0,102.30,0.01
4642308,5065.0,10130.0,2.0,101.30,0.01
1977319,4902.5,9805.0,2.0,98.05,0.01
4969553,4848.0,9696.0,2.0,96.96,0.01
5310851,4790.0,9580.0,2.0,95.80,0.01
6000206,4668.0,9336.0,2.0,93.36,0.01
123669,4548.0,9096.0,2.0,90.96,0.01


## Grouping

The core of grouping is a simple idea, but it has profound implications. It means that we can measure different parts of our data in a single query, producing a data frame that can itself then be analyzed, sorted, and displayed. 

TODO 
- For each number of passengers, find the mean cost of a taxi ride. Sort this result from lowest (i.e., cheapest) to highest (i.e., most expensive).

In [78]:
taxi_df.groupby('passenger_count').mean()["total_amount"].sort_values(ascending = False)

passenger_count
9.0    81.244211
7.0    71.143103
8.0    58.197059
4.0    19.128092
2.0    19.050504
3.0    18.736862
6.0    18.367962
1.0    18.343110
5.0    18.234443
0.0    18.059724
Name: total_amount, dtype: float64

TODO
- Sort the results once again, in increasing number of passengers.

In [79]:
taxi_df.groupby('passenger_count').mean()["trip_distance"].sort_index()

passenger_count
0.0    2.689548
1.0    2.811050
2.0    3.001117
3.0    2.930363
4.0    2.980372
5.0    2.850356
6.0    2.906041
7.0    3.589655
8.0    2.960000
9.0    3.314737
Name: trip_distance, dtype: float64

TODO
- Now create a new column, `trip_distance_group`, in which the values will be short (< 2 miles), medium (>= 2 miles and ⇐ 10 miles), or long (> 10 miles). What was the average number of passengers per trip length category? Sort this result from highest (greatest number of passengers) to lowest (smallest number of passengers).

In [82]:
taxi_df["trip_distance_group"] = pd.cut(taxi_df["trip_distance"], bins = [taxi_df["trip_distance"].min(), 2, 10, taxi_df["trip_distance"].max()], labels = ["short", "medium", "long"])

# Now that we have a new column lets group by this
taxi_df.groupby('trip_distance_group')["passenger_count"].mean().sort_values(ascending = False)

trip_distance_group
long      1.547586
medium    1.532448
short     1.503263
Name: passenger_count, dtype: float64

TODO
- Create a single data frame containing rides from both January 2019 and January 2020, with a column year indicating which year it came from. Use groupby to compare the average cost of a taxi in January of each of these two years.

In [95]:
# Load the data 2020-01 data file
taxi_df_1 = pd.read_csv('data/nyc_taxi_2020-01.csv', usecols = ['total_amount', 'trip_distance', "tpep_pickup_datetime"], parse_dates = ['tpep_pickup_datetime'])

# Create an new year_month column from the date
taxi_df_1["year_month"] = taxi_df_1["tpep_pickup_datetime"].dt.to_period("M")


# Load the data 2020-07 data file
taxi_df_2 = pd.read_csv('data/nyc_taxi_2020-07.csv', usecols = ['total_amount', 'trip_distance', "tpep_pickup_datetime"], parse_dates = ['tpep_pickup_datetime'])

# Create an new year_month column from the date
taxi_df_2["year_month"] = taxi_df_2["tpep_pickup_datetime"].dt.to_period("M")


# Join the 2 dataframes
main_taxi_df = pd.concat([taxi_df_1, taxi_df_2]) 

# Now Group by the year-month and compare average taxi fare
main_taxi_df.groupby('year_month')["total_amount"].mean().sort_values(ascending = False)

year_month
2020-02    30.836000
2008-12    25.504545
2009-01    22.575000
2020-08    21.380000
2020-06    20.300000
2020-01    18.663096
2020-07    18.631421
2020-05    18.220000
2019-12    17.802977
2021-01    15.920000
2020-03    14.900000
2020-04    10.560000
2003-01     0.000000
Freq: M, Name: total_amount, dtype: float64

## Joining

Like grouping, joining is a concept that you might have encountered previously, when working with relational databases. The joining functionality in pandas is quite similar to that sort of database, although the syntax is quite different.

Separating your data into two or more pieces, so that each piece of information appears only a single time, is known as "normalization." There are all sorts of formal theories and descriptions of normalization, but it all boils down to keeping the information in separate places, and joining data frames when necessary.

Sometimes, you’ll normalize your own data. But sometimes, you’ll receive data that has been normalized, and then separated into separate pieces. For example, many data sets are distributed in separate CSV file, which almost always means that you’ll need to join two or more data frames together in order to analyze the information. Other times, you might want to normalize the data yourself, in order to gain flexibility or performance.

TODO

- Load the OECD tourism data (from oecd_tourism.oecd) into a data frame. We’re interested in the following columns:

    -   LOCATION, a three-letter abbreviation for the country name
    -   SUBJECT, either INT_REC (for tourist funds received) or INT-EXP (for tourist expenses).
    -   TIME, a year (integer)
    -   Value, a float indicating thousands of dollars.

- Find the five countries that received the greatest amount of tourist dollars, on average, accross years in the data set.

In [101]:

oecd_df = pd.read_csv('data/oecd_tourism.csv', usecols = ['LOCATION', 'SUBJECT', "TIME", "Value"])


oecd_df.loc[(oecd_df["SUBJECT"] == "INT_REC")].groupby("LOCATION")["Value"].mean().sort_values(ascending = False).iloc[:5]

LOCATION
USA    201613.500000
ESP     69655.817364
FRA     65063.335727
DEU     53408.570636
GBR     51752.090909
Name: Value, dtype: float64

TODO
- Find the first five countries whose citizens spent the least amount of tourist dollars, on average, across years 

In [103]:
oecd_df.loc[(oecd_df["SUBJECT"] == "INT-EXP")].groupby("LOCATION")["Value"].mean().sort_values(ascending = True).iloc[:5]

LOCATION
MLT     387.801667
CRI     867.075000
LVA     919.545455
ISL    1072.819636
HRV    1115.628083
Name: Value, dtype: float64

In [106]:
oecd_locations = pd.read_csv('data/oecd_locations.csv', header = None ,names = ["Country_Code", "Country"], index_col = "Country_Code")

oecd_locations.head()

,Country
Country_Code,
AUS,Australia
AUT,Austria
BEL,Belgium
CAN,Canada
DNK,Denmark


Now we’ll bring this all together: I’ll create a new data frame, the result of joining locations_df and tourism_df. The problem is that while the three-letter abbreviation (i.e., LOCATION) is the index of locations_df, it’s just a plain ol' column in tourism_df. And yes, you can join on non-index columns in pandas, but it makes the code a bit shorter and clearer to have the data frames share index values.

In [107]:
oecd_full_df = oecd_df.set_index("LOCATION").join(oecd_locations)

oecd_full_df.head()

,SUBJECT,TIME,Value,Country
AUS,INT_REC,2008,31159.8,Australia
AUS,INT_REC,2009,29980.7,Australia
AUS,INT_REC,2010,35165.5,Australia
AUS,INT_REC,2011,38710.1,Australia
AUS,INT_REC,2012,38003.7,Australia


## Multi-city Temperatures

Take the eight CSV files containing weather data that provided, from eight different cities (spanning four states), and turn them into a data frame:

- The files are: san+francisco,ca.csv, new+york,ny.csv, springfield,ma.csv, boston,ma.csv, springfield,il.csv, albany,ny.csv, los+angeles,ca.csv, and chicago,il.csv.
- We are only interested in the first three columns from each CSV file, namely the `date and time`, the `max temperature`, and the `min temperature`.
- Add `city` and `state` columns, which will contain the city and state from the filename, and will allow us to distinguish between rows

In [15]:
import glob

import pandas as pd

all_dfs = []


for one_file in glob.glob("data/*,*.csv"):
    print(f"Loading {one_file}")
    # Grab the city and state from the filename
    city, state = one_file.removeprefix("data\\").removesuffix(".csv").split(",")
    # Read the csv file and add the city and state to the dataframe
    one_df = pd.read_csv(one_file, usecols = [0,1,2], names = ["date_time", "max_temp", "min_temp"], header = 0)
    one_df['city'] = city.replace("+", " ")
    one_df['state'] = state.upper()
    all_dfs.append(one_df)

# Combine all the dfs
df = pd.concat(all_dfs)

df.head()

Loading data\albany,ny.csv
Loading data\boston,ma.csv
Loading data\chicago,il.csv
Loading data\los+angeles,ca.csv
Loading data\new+york,ny.csv
Loading data\san+francisco,ca.csv
Loading data\springfield,il.csv
Loading data\springfield,ma.csv


,date_time,max_temp,min_temp,city,state
0,2018-12-11 00:00:00,-2,-8,albany,NY
1,2018-12-11 03:00:00,-2,-8,albany,NY
2,2018-12-11 06:00:00,-2,-8,albany,NY
3,2018-12-11 09:00:00,-2,-8,albany,NY
4,2018-12-11 12:00:00,-2,-8,albany,NY


- Does the data for each city and state start at roughly the same time?
 

In [17]:
df.groupby(["state", "city"])["date_time"].min().sort_values()

state  city         
CA     los angeles      2018-12-11 00:00:00
       san francisco    2018-12-11 00:00:00
IL     chicago          2018-12-11 00:00:00
       springfield      2018-12-11 00:00:00
MA     boston           2018-12-11 00:00:00
       springfield      2018-12-11 00:00:00
NY     albany           2018-12-11 00:00:00
       new york         2018-12-11 00:00:00
Name: date_time, dtype: object

> In the above code, I tell pandas that I want to get the minimum value of date_time for each distinct combination of state and city. I then want to sort the values, so that I can easily find the earliest one—as well as find out if they’re all from the same period of time. I can similarly run max on the values, to find the highest one:

In [18]:
df.groupby(["state", "city"])["date_time"].max().sort_values()

state  city         
CA     los angeles      2019-03-11 21:00:00
       san francisco    2019-03-11 21:00:00
IL     chicago          2019-03-11 21:00:00
       springfield      2019-03-11 21:00:00
MA     boston           2019-03-11 21:00:00
       springfield      2019-03-11 21:00:00
NY     albany           2019-03-11 21:00:00
       new york         2019-03-11 21:00:00
Name: date_time, dtype: object

- Find the lowest minimum temperature recorded for each city in our data set. 

In [19]:
df.groupby(["state", "city"])["min_temp"].min().sort_values()

state  city         
IL     chicago         -28
       springfield     -25
MA     springfield     -20
NY     albany          -19
MA     boston          -14
NY     new york        -14
CA     san francisco     3
       los angeles       4
Name: min_temp, dtype: int64

- Find the highest maximum temperature recorded during this period, but on a per-state basis.

In [20]:
df.groupby(["state"])["max_temp"].max().sort_values()

state
NY    15
IL    16
MA    17
CA    23
Name: max_temp, dtype: int64

### Window Functions 



In [22]:
df = pd.DataFrame({'sales':[100, 150, 200, 250,
                200, 150, 300, 400,
                500, 100, 300, 200],
              'quarters':'Q1 Q2 Q3 Q4'.split() * 3})

df.head()

,sales,quarters
0,100,Q1
1,150,Q2
2,200,Q3
3,250,Q4
4,200,Q1


But what if we want to find out how much we sold, total, through the current quarter? That is, I want to know how much we sold in Q1. Then in Q1+Q2. Then Q1+Q2+Q3. And so on, until the final result will be df['sales'].sum().

To perform this kind of operation, pandas provides us with "window functions." There are several different types of window functions, but the basic idea is that they allow us to run an aggregate function, such as mean, on subsections of our data frame.

What I described earlier, that we would like to know, for each quarter, how much we revenue we had through that quarter, is a classic example of a window function. This is known as an "expanding window," because we run the function with an ever-expanding number of lines—first one line, then two, then three… all the way up to the entire data frame.

In [23]:
df["sales"].expanding().sum()

0      100.0
1      250.0
2      450.0
3      700.0
4      900.0
5     1050.0
6     1350.0
7     1750.0
8     2250.0
9     2350.0
10    2650.0
11    2850.0
Name: sales, dtype: float64

This returns a series whose values are the cumulative sum of values in sales up to that point. Since the first four values in the sales column are 100, 150, 200, and 250, the output of our call to expanding will be 100, 250, 450, and 700.

Perhaps we don’t want to get a cumulative total, but rather want to get a running average of how much we’ve sold per quarter. We can run mean, or any other aggregation method:

In [24]:
df["sales"].expanding().mean()

0     100.000000
1     125.000000
2     150.000000
3     175.000000
4     180.000000
5     175.000000
6     192.857143
7     218.750000
8     250.000000
9     235.000000
10    240.909091
11    237.500000
Name: sales, dtype: float64

In this case, the output from expanding will be 100, 125, 150, and 175.

We can also use a "rolling" window function. In this case, we determine how many rows will be considered to be part of the window. For example, if the window size is 3, then we’ll run the aggregation function on row index 0-2, then 1-3, then 2-4, etc., until we get to the end of the data frame. For example, if you want to find out the mean of rows that are close to one another, you can do it as follows:

In [26]:
df["sales"].rolling(3).mean()

0            NaN
1            NaN
2     150.000000
3     200.000000
4     216.666667
5     200.000000
6     216.666667
7     283.333333
8     400.000000
9     333.333333
10    300.000000
11    200.000000
Name: sales, dtype: float64

In the above code, rolling is how I indicate that I want to run a rolling window function, and the argument 3 indicates that I want to have three rows in each window. We’ll thus invoke mean on rows 0-2, then 1-3, then 2-4, then 3-5, etc. The series that we get back from this call will put the result of mean in the same location as the third (and final) row in our rolling window. This means that row indexes 0 and 1 will have NaN values.

> A third type of window function is `pct_change`. When we run this on a series, we get back a new series, with NaN at row index 0. The remaining rows indicate the percentage change from the previous row to the current one:

In [27]:
df["sales"].pct_change()

0          NaN
1     0.500000
2     0.333333
3     0.250000
4    -0.200000
5    -0.250000
6     1.000000
7     0.333333
8     0.250000
9    -0.800000
10    2.000000
11   -0.333333
Name: sales, dtype: float64

The result is calculated as (later_row - earlier_row) / earlier_row:

index 0 is always NaN
index 1 is the result of calculating (150 - 100) / 100
index 2 is the result of calculating (200 - 150) / 150
index 3 is the result of calculating (250 - 200) / 200
pct_change is great for finding how much your values have gone up, or down, from row to row

In [2]:
import pandas as pd

filename = "data/sat-scores.csv"

df = pd.read_csv(filename,
                usecols=['Year', 'State.Code', 'Total.Math',
                         'Family Income.Less than 20k.Math',
                         'Family Income.Between 20-40k.Math',
                         'Family Income.Between 40-60k.Math',
                         'Family Income.Between 60-80k.Math',
                         'Family Income.Between 80-100k.Math',
                         'Family Income.More than 100k.Math'])

df.head()

,Year,State.Code,Total.Math,Family Income.Between 20-40k.Math,Family Income.Between 40-60k.Math,Family Income.Between 60-80k.Math,Family Income.Between 80-100k.Math,Family Income.Less than 20k.Math,Family Income.More than 100k.Math
0,2005,AL,559,513,539,550,566,462,588
1,2005,AK,519,492,517,513,528,464,541
2,2005,AZ,530,498,520,524,534,485,554
3,2005,AR,552,513,543,553,570,489,572
4,2005,CA,522,477,506,521,535,451,566


In [4]:
# Rename the Columns

df.columns = ['Year', 'State.Code', 'Total.Math',
                      'income<20k',
                      '20k<income<40k',
                      '40k<income<60k',
                      '60k<income<80k',
                      '80k<income<100k',
                      'income>100k',
                      ]

df.head()



,Year,State.Code,Total.Math,income<20k,20k<income<40k,40k<income<60k,60k<income<80k,80k<income<100k,income>100k
0,2005,AL,559,513,539,550,566,462,588
1,2005,AK,519,492,517,513,528,464,541
2,2005,AZ,530,498,520,524,534,485,554
3,2005,AR,552,513,543,553,570,489,572
4,2005,CA,522,477,506,521,535,451,566


So long as the assigned list of strings contains the same number of elements as df has columns, this assignment will work just fine.

Now that our data frame has the rows and columns that we want, and that the columns have easy-to-understand names, we can start to actually analyze things.

First, I asked you to find the average SAT math score for each income level, grouped and then sorted by year:

In [5]:
df.groupby("Year").mean().sort_index()

,Total.Math,income<20k,20k<income<40k,40k<income<60k,60k<income<80k,80k<income<100k,income>100k
Year,,,,,,,
2005,535.653846,488.653846,522.673077,536.076923,548.942308,427.596154,572.173077
2006,537.480769,502.923077,523.769231,534.903846,550.461538,461.019231,572.519231
2007,535.339623,494.849057,519.490566,533.188679,545.698113,457.924528,565.169811
2008,535.981132,523.622642,547.471698,549.188679,557.641509,478.641509,564.566038
2009,540.803922,527.823529,550.980392,553.941176,565.333333,482.058824,585.784314
2010,540.843137,499.274510,522.000000,534.235294,547.627451,477.039216,569.274510
2011,533.226415,494.886792,513.415094,528.660377,541.849057,460.452830,563.245283
2012,533.603774,492.056604,512.452830,525.773585,538.301887,458.773585,557.320755
2013,532.622642,490.132075,511.377358,520.320755,537.396226,469.358491,556.339623


This query is similar to what we’ve done before: We want to invoke mean on every column in df, grouping the results by year. We’ll thus be able to say, for each income bracket, what the average SAT math score was across the United States in each year.

Because we’re grouping by the Year column, it won’t be included in our output. But why wasn’t State.Code included in our output? Because it’s a textual column, and mean only works on numeric columns. As a result, we didn’t need to explicitly exclude it—or explicitly indicate which columns we did want—in our output.

Moreover, because we grouped by Year, the index of the resulting data frame had an index of Year. It so happens that because the data set come sorted by Year that the results appear to be sorted. But just to be on the safe side, I invoked sort_index on the data frame, ensuring that the result we got back was sorted, from the earliest year in the data set through the final year in the data set.

We want to compare the scores by year and income brackets. But pct_change works on rows, not on columns—and right now, our data frame has the brackets as columns. We thus need to flip the data frame on its side, such that the years will be the columns and the income brackets will be the columns.

The solution is to use the transpose method, more easily abbreviated as T, which returns a new data frame in which the rows and columns have exchanged places:

In [8]:
df.groupby('Year')[['income<20k',
                      '20k<income<40k',
                      '40k<income<60k',
                      '60k<income<80k',
                      '80k<income<100k',
                      'income>100k']].mean().T.pct_change() * 100

Year,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
income<20k,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20k<income<40k,6.961826,4.144998,4.979601,4.554627,4.387236,4.551702,3.743948,4.145098,4.334604,3.476777,4.505853
40k<income<60k,2.564480,2.125863,2.636836,0.313620,0.537367,2.343926,2.969387,2.599411,1.748884,2.374322,2.603841
60k<income<80k,2.399914,2.908503,2.346155,1.539149,2.056564,2.506790,2.494736,2.382832,3.281720,3.027917,2.827690
80k<income<100k,-22.105448,-16.248603,-16.084641,-14.166808,-14.730161,-12.889828,-15.021937,-14.773922,-12.660628,-15.413743,-17.442913
income>100k,33.811558,24.185542,23.419860,17.951750,21.517185,19.334950,22.324209,21.480568,18.531918,20.900242,25.909685


From a visual scan of the data, we can see that nearly each income bracket did better than the next-lower bracket. Thus, families with an income between $20,000 and $40,000 per year, did about 3 to 7 percent better on their math SAT than people in the lowest bracket. And in families making $40,000 to $60,000 per year, they generally did 2-3 percent better than those in the next-lower bracket.

However, we also see that across the years, those earning between $80,000 and $100,000 per year did slightly worse than those than those in the next-lowest income bracket (i.e., between $60,000 and $80,000 per year). What’s the reason for this? I’m not at all sure, but we see that this is consistently true across all of the years.

>  Find which income bracket, on average, had the greatest advantage over the next-poorer income bracket. In order to do this, I started with the result of our call to pct_change. But I wanted to find out how much better, on average, each bracket did than the next-poorer bracket. To do this, I would want to use mean—but not on the data frame we got back from pct_change. Rather, I want to re-transpose the data frame, such that the income brackets are the columns, and the years are the rows:

In [13]:
change = df.groupby('Year')[['income<20k',
                      '20k<income<40k',
                      '40k<income<60k',
                      '60k<income<80k',
                      '80k<income<100k',
                      'income>100k']].mean().T.pct_change().T.mean().sort_values(ascending = False)


change[change <= 0]

80k<income<100k   -0.155944
dtype: float64

### Filtering and Transforming


#### Filtering

In [14]:
import numpy as np 

np.random.seed(0)

df = pd.DataFrame({"name": list("ABCDEFGHIJ"),
                   "year": [2018, 2019, 2020] * 3 +[2021],
                   "score": np.random.randint(80, 100, 10)
        })

df.head()

,name,year,score
0,A,2018,92
1,B,2019,95
2,C,2020,80
3,D,2018,83
4,E,2019,83


In [15]:
# Get the mean score

df["score"].mean()

89.0

In [16]:
# All students who scored above 90

df.loc[(df["score"] > 90)]

,name,year,score
0,A,2018,92
1,B,2019,95
7,H,2019,99
8,I,2020,98


In [17]:
# Mean Score over year

df.groupby('year')["score"].mean()

year
2018    88.000000
2019    92.333333
2020    88.333333
2021    84.000000
Name: score, dtype: float64

> So far, so good. But consider this: I want to find out which years in our school had an average score of at least 90, and see all of the students in those years. In other words, I want to filter out specific groups fo students, based on a per-year aggregate calculation. How can I do that?

In [18]:
def year_average_is_at_least_90(df):
    return df["score"].mean() >= 90

df.groupby("year").filter(year_average_is_at_least_90)

,name,year,score
1,B,2019,95
4,E,2019,83
7,H,2019,99


The result of running this code will be a data frame whose rows all come from df, from years in which the average final-exam math score was at least 90. That would only be in the year 2019, so we get the rows with indexes 1, 4, and 7.

In [2]:
import glob

import pandas as pd

all_dfs = []


for one_file in glob.glob("data/*,*.csv"):
    print(f"Loading {one_file}")
    # Grab the city and state from the filename
    city, state = one_file.removeprefix("data\\").removesuffix(".csv").split(",")
    # Read the csv file and add the city and state to the dataframe
    one_df = pd.read_csv(one_file, usecols = [1,2,19], names = ["max_temp", "min_temp", "precipMM"], header = 0)
    one_df['city'] = city.replace("+", " ")
    one_df['state'] = state.upper()
    all_dfs.append(one_df)

# Combine all the dfs
df = pd.concat(all_dfs)

# Preview the DF
df.head()

Loading data\albany,ny.csv
Loading data\boston,ma.csv
Loading data\chicago,il.csv
Loading data\los+angeles,ca.csv
Loading data\new+york,ny.csv
Loading data\san+francisco,ca.csv
Loading data\springfield,il.csv
Loading data\springfield,ma.csv


,max_temp,min_temp,precipMM,city,state
0,-2,-8,0.0,albany,NY
1,-2,-8,0.0,albany,NY
2,-2,-8,0.0,albany,NY
3,-2,-8,0.0,albany,NY
4,-2,-8,0.0,albany,NY


For starters, I wanted to find cities which had measured precipitation of 15 mm or more on at least three occasions. This means:

- We’ll need to group our data frame by city
- We’ll check to see which cities had 15 mm of precipitation at least three times

In [6]:
def has_multiple_readings_at_least(df):
    return df.loc[(df["precipMM"] > 15), ["precipMM"]].count() > 3


df.groupby(["city", "state"]).filter(has_multiple_readings_at_least)[["city", "state"]]

,city,state
0,los angeles,CA
1,los angeles,CA
2,los angeles,CA
3,los angeles,CA
4,los angeles,CA
...,...,...
723,new york,NY
724,new york,NY
725,new york,NY
726,new york,NY


The way filter on a DataFrameGroupBy object works, the check will be done with a function. The function will return True (indicating that the group passed the criteria) or False (indicating that it did not). Rows from groups that passed will be returned in the final data frame.

Since we want to find the precipitation on a per-city basis, you might think that we should group by city name:
`df.groupby('city')`

However, we can’t do this, because there are two different cities with the name "Springfield"—both in Illinois and Massachusetts. For that reason, we’ll need to group not just by city, but also by state. We can do that, of course, by passing a list of columns to `groupby`, rather than just a single column:

`df.groupby(['city', 'state'])`

This gives us our groupby object, which we’ve previously used to apply aggregate functions on distinct subsets of our data. But here, we’re going to use the Groupby object in a different way, to include and exclude rows from df based on properties of their city and state. That is, I want to filter out rows, but I want to do it by group—such that for each group, all of the rows are included or excluded. (You can think of this as the collective punishment feature of pandas.)

We do this by calling filter on our GroupBy object. Whereas filter on a data frame works on a row-by-row basis, filter on a GroupBy works on a group-by-group basis. The argument to filter is a function, one which expects to get a data frame as its argument. The function will be called once for each group in the GroupBy, and the data frame passed to it will be a subset of the original data frame, containing only those rows in the current group.

The function passed to filter should return True or False. If the function returns True, then the rows from this sub-frame will be kept. If the function returns False, then the rows from this sub-frame will not be included. Because its argument is a data frame with all of the rows in the current group, filter can perform all sorts of calculations in determining whether to return True or False.

In our case, we want to preserve rows from cities that had 15 mm of precipitation on at least three occasions. Our function will thus need to determine whether the sub-frame it is passed contains at least three such rows. 

In [7]:
def has_multiple_readings_at_least(df):
    return df.loc[(df["precipMM"] > 15), ["precipMM"]].count() > 3


output = df.groupby(["city", "state"]).filter(has_multiple_readings_at_least)[["city", "state"]]

output.set_index(["city", "state"]).index.unique()

MultiIndex([('los angeles', 'CA'),
            (   'new york', 'NY')],
           names=['city', 'state'])

This works, and gives us the answer we wanted—namely, that only New York and Los Angeles had three occasions on which at least 15 mm of precipitation fell. However, if you’ve been programming for any length of time, the has_multiple_readings_at_least function might have seemed a bit odd. Do we really want to hard-code the values of 15 mm and 3 times into the function? It might make more sense to write a more generic function, one which can take additional arguments.

But how can we do that? After all, we’re not calling has_multiple_readings_at_least directly. Rather, we’re passing it to the filter method, which calls the function on our behalf. And there isn’t an obvious way for us to pass arguments to our function when it’s being invoked via filter.

Here, pandas does something clever: Any additional arguments passed to filter are passed along to our function. This is done using the standard Python constructs of *args and **kwargs, for arbitrary positional and keyword arguments. This works, and gives us the answer we wanted—namely, that only New York and Los Angeles had three occasions on which at least 15 mm of precipitation fell. However, if you’ve been programming for any length of time, the has_multiple_readings_at_least function might have seemed a bit odd. Do we really want to hard-code the values of 15 mm and 3 times into the function? It might make more sense to write a more generic function, one which can take additional arguments.

But how can we do that? After all, we’re not calling has_multiple_readings_at_least directly. Rather, we’re passing it to the filter method, which calls the function on our behalf. And there isn’t an obvious way for us to pass arguments to our function when it’s being invoked via filter.

Here, pandas does something clever: Any additional arguments passed to filter are passed along to our function. This is done using the standard Python constructs of *args and **kwargs, for arbitrary positional and keyword arguments. This works, and gives us the answer we wanted—namely, that only New York and Los Angeles had three occasions on which at least 15 mm of precipitation fell. However, if you’ve been programming for any length of time, the has_multiple_readings_at_least function might have seemed a bit odd. Do we really want to hard-code the values of 15 mm and 3 times into the function? It might make more sense to write a more generic function, one which can take additional arguments.

But how can we do that? After all, we’re not calling has_multiple_readings_at_least directly. Rather, we’re passing it to the filter method, which calls the function on our behalf. And there isn’t an obvious way for us to pass arguments to our function when it’s being invoked via filter.

Here, pandas does something clever: Any additional arguments passed to filter are passed along to our function. This is done using the standard Python constructs of *args and **kwargs, for arbitrary positional and keyword arguments. 

> We can thus rewrite the function as

In [8]:
def has_multiple_readings_at_least(df, min_mm, times):
    return df.loc[(df["precipMM"] > min_mm), ["precipMM"]].count() > times


Now it looks more like a regular Python function, taking three arguments. The first will still be the sub-frame that was passed before, containing all of the rows in the current group. But the second two arguments will be assigned values based on either the additional positional arguments passed to filter or the additional keyword arguments:

In [9]:
output = df.groupby(['city', 'state']).filter(
    has_multiple_readings_at_least,
    min_mm=10, times=3)[['city', 'state', 'precipMM']]

output.set_index(['city', 'state']).index.unique()

MultiIndex([(       'albany', 'NY'),
            (       'boston', 'MA'),
            (  'los angeles', 'CA'),
            (     'new york', 'NY'),
            ('san francisco', 'CA')],
           names=['city', 'state'])

In the above code, you can see that we’re calling filter, and passing it our function, has_multiple_readings_at_least. In theory, we could then pass values for min_mm and times as positional arguments. But if we do that, we’ll also have to pass a second positional argument to filter, called dropna. Rather than calling filter(func, True, 10, 3), I decided to call filter(func, min_mm=10, times=3). This is an aesthetic choice, rather than a technical one, but I think it makes sense in this case

#### Transform

The next part of this exercise asked you to find the proportion of that city’s precipitation that fell with each measurement. If our data frame contains two precipitation measurements for a given city, and we see that 3 mm fell on the first day, while 7 mm fell on the second day, I’d want to find that 30% fell in the first measurement, and 70% fell in the second.

In other words, we’re going to calculate one value for each row. But the value we calculate for each row will depend on an aggregate calculation for the row’s group. It’s precisely for these situations that pandas provides us with a Groupby transform method. Similar to what we did with filter, we’ll pass a function as the first argument to transform. This function will be invoked once per group, and the function will be passed a series—the column that we want to transform. The function must then return a series, of the same length and with the same index, as its argument.

Let’s assume that we have a series of numbers, each representing one measurement of precipitation. What function could I write that would return a new series, one with the same length and index as the original, but whose values would indicate the proportion of the whole? It might look like this:

In [11]:
def proportion_of_city_prep(s):
    return s/s.sum()

Our function takes a series `s` as input, and then returns the result of dividing each row by the sum total of all rows. This is how we would do it if all of the values were from the same city. How can we do it, then, if we have many different cities? That’s part of the magic—the `groupby` transform method takes care of that for us. The rows from each group are passed, one at a time, to the function proportion_of_city_precip. The return value is then a series in which the parallel rows from the input series have their new values. We can assign the resulting series back to the column from which it was transformed, add a new column to a data frame, or just save the transformed column.

The difference between the standard transform method and Groupby’s transform is that in the latter, we have access to the entire series, and can thus make calculations using aggregation functions.

Here’s how we would use our proportion_of_city_precip function along with Groupby’s transform:

In [14]:
df["precip_pct"] = df.groupby("city")["precipMM"].transform(proportion_of_city_prep)

df.head(10)

,max_temp,min_temp,precipMM,city,state,precip_pct
0,-2,-8,0.0,albany,NY,0.000000
1,-2,-8,0.0,albany,NY,0.000000
2,-2,-8,0.0,albany,NY,0.000000
3,-2,-8,0.0,albany,NY,0.000000
4,-2,-8,0.0,albany,NY,0.000000
5,-2,-8,0.0,albany,NY,0.000000
6,-2,-8,0.0,albany,NY,0.000000
7,-2,-8,0.0,albany,NY,0.000000
8,0,-5,0.0,albany,NY,0.000000
9,0,-5,0.1,albany,NY,0.000223


In [15]:
df.groupby(["city", "state"])["precip_pct"].max()

city           state
albany         NY       0.029228
boston         MA       0.048302
chicago        IL       0.057257
los angeles    CA       0.059242
new york       NY       0.055149
san francisco  CA       0.056509
springfield    IL       0.030977
               MA       0.023459
Name: precip_pct, dtype: float64

### Wine Scores and Tourism Spending



In [16]:
oecd_df = pd.read_csv("data/oecd_locations.csv", header = None, names = ["abbrev", "country"], index_col = "abbrev")


oecd_tourism_df = pd.read_csv("data/oecd_tourism.csv", usecols = ["LOCATION", "TIME", "Value"], index_col = "LOCATION")

# Tourist Spending per country
oecd_tourism_df.groupby("LOCATION")["Value"].mean()

LOCATION
AUS     37634.433333
AUT     16673.886364
BEL     16525.237545
BGR      2953.091583
BRA     13942.913958
CAN     32593.612500
CHE     16128.504167
CHL      2786.516667
CHN    103385.856917
COL      4580.304167
CRI      2020.587500
CZE      6404.100409
DEU     75011.823091
DNK     10362.563636
EGY      6526.425792
ESP     45628.886318
EST      1485.284636
FIN      5288.658591
FRA     58228.804000
GBR     63507.159091
GRC     10197.792909
HRV      5201.199042
HUN      5108.871591
IDN     10771.700833
IND     18828.668583
IRL      8782.930591
ISL      1735.093545
ISR      6634.454042
ITA     39539.560000
JPN     28606.891667
KAZ      2293.282708
KOR     21677.131818
LTU      1308.044444
LUX      4238.920682
LVA      1067.290909
MAR      5491.630625
MEX     14016.690909
MLT       997.735458
NLD     21160.718182
NOR     10428.327273
NZL      7146.183333
PER      2916.227625
PHL      7435.858042
POL     10718.590773
PRT     10754.012045
ROU      2733.599042
RUS     27854.359091
SRB 

The above code says that we want to get the mean of the Value column for each distinct LOCATION. (Notice that even though LOCATION is now the index of this data frame, we can still use it for grouping.) However, we don’t want LOCATION, containing the country abbreviations. Rather, we want to use the country names, which are in oecd_df.

We’ll thus need to join these two data frames together. Both use the abbreviations as an index, which makes this possible. (It doesn’t matter that the columns have different names; joining typically works on the data frames' indexes.) When we join, we basically say that we want to create a new, wider data frame containing all of the columns from the first and all of the columns of the second, with the indexes overlapping. So the resulting data frame will have a total of four columns: An index containing the location abbreviations, as before, a country column (from oecd_df), and TIME and Value columns (from oecd_tourism_df). The left and right sides will be joined together wherever the index of oecd_df matches the index of oecd_tourism_df, which means that it’s not a problem to have repeated values in the indexes of one or both data frames.

In [19]:
# Join the 2 dataframes
tourism_spending = oecd_df.join(oecd_tourism_df).groupby("country")["Value"].mean()

In this way, we’ve again calculated and retrieved the mean tourism spending, per country, over all years in the data set. But the result that we get back uses the full country names, rather than the abbreviations. Moreover, because the result has an index (country names) and a single value column, it’s returned to us as a series, rather than as a data frame. I asked you to assign the resulting series to a variable, tourism_spending, for easier manipulation later on:

In [22]:
wine_df = pd.read_csv("data/winemag-150k-reviews.csv", usecols = ["country", "points"])



country_points = wine_df.groupby("country")["points"].mean().sort_values(ascending = False)

We get back a new series showing which countries had the highest average wine scores, and which had the lowest.

But now we come to the climax of this exercise: I want to join together the wine scores and the tourism spending. How can I do that?

Well, it makes sense that I’d want to use join again, with country_points on the left (i.e., as the data frame on which we invoke join) and with tourism_spending on the right (i.e., as the data frame passed as an argument to join). There’s just one problem with this, namely that country_points is a series, and you can only invoke join on a data frame. (You can pass a series as the argument to join, though—so a series can be the right side, but not the left side, of a pandas join.)

Fortunately, we can call the `to_frame` method on our series, and get back a single-column data frame with the same index as we had in the series:

In [25]:
country_points.to_frame().join(tourism_spending, how = "outer")

,points,Value
country,,
Albania,88.000000,NaN
Argentina,85.996093,NaN
Australia,87.892475,37634.433333
Austria,89.276742,16673.886364
Belgium,NaN,16525.237545
Bosnia and Herzegovina,84.750000,NaN
Brazil,83.240000,13942.913958
Bulgaria,85.467532,NaN
Canada,88.239796,32593.612500


Once again, it’s important to remember that a join links the left data frame with the right one, connecting them along their indexes. In this case, we’ll end up with three columns: country, the index column that is shared by the left and right, points from the left, and Value from the right